In [1]:
import shutil
import os
import sys
from pathlib import Path
import multiprocessing
import subprocess
from osgeo import gdal
import numpy as np

In [2]:
def progress_callback(complete, message, unknown):
    print('progress: {}, message: "{}", unknown {}'.format(complete, message, unknown))
    return 1

In [3]:
path = "/home/jovyan/work/satellite_data/ask_new/"

In [4]:
parts = {i: os.path.join(path,i) for i in os.listdir(path) if os.path.isdir(os.path.join(path,i))}

In [5]:
file_list = []
for part,path_part in parts.items():
    f = os.listdir(path_part)
    
    subpath = os.path.join(path,part)
    out_file = os.path.join(subpath,"cog_"+str(part)+".tif")
    
    if os.path.basename(out_file) in f:
        file_list.append(out_file)

# Variant 1

In [ ]:
tmp_file = os.path.join(path,"tmp_all.tif")
out_file = os.path.join(path,"all_cog.tif")
warp_options = "-overwrite -multi -wm 80% -t_srs WGS84 -srcnodata 0 -dstnodata 0 -co TILED=YES \
                -co COMPRESS=DEFLATE -co BIGTIFF=YES -wo NUM_THREADS=ALL_CPUS -co NUM_THREADS=ALL_CPUS" #-co BLOCKSIZE=256"
cog_options = "-co BIGTIFF=YES -co NUM_THREADS=ALL_CPUS -co COMPRESS=DEFLATE --config GDAL_CACHEMAX 20096 -co PREDICTOR=2" 
gdal.SetConfigOption('GDAL_CACHEMAX', '20096')
#### Transform in lossless format + WGS84 format
ds = gdal.Warp(tmp_file, file_list, format="GTiff",options=warp_options,callback = progress_callback)
ds = None
del ds

cmd = ['gdal_translate', '-of','COG'] + cog_options.split(" ") + [tmp_file,out_file]
process = subprocess.Popen(cmd)
try:
    process.wait(timeout=None) #one day
except subprocess.TimeoutExpired:
    print('Timed out - killing', process.pid)
    process.kill()

os.remove(tmp_file)

progress: 0.0, message: "Processing /home/jovyan/work/satellite_data/ask_new/6179/cog_6179.tif [1/355]", unknown None


# Variant 2

In [6]:
vrt_name = os.path.join(path,"all.vrt")

ds = gdal.BuildVRT(vrt_name,file_list)
ds = None

In [7]:
cog_options = "-co BIGTIFF=YES -co NUM_THREADS=ALL_CPUS -co COMPRESS=DEFLATE --config GDAL_CACHEMAX 8096 -co PREDICTOR=2" 
gdal.SetConfigOption('GDAL_CACHEMAX', '8096')

In [ ]:
out_file = os.path.join(path,"all_cog.tif")
cmd = ['gdal_translate', '-of','COG'] + cog_options.split(" ") + [vrt_name,out_file]
process = subprocess.Popen(cmd)
try:
    process.wait(timeout=None) #one day
except subprocess.TimeoutExpired:
    print('Timed out - killing', process.pid)
    process.kill()

Input file size is 4486338, 2018483
0